In [1]:
# 

In [2]:
import sys 
sys.path.append('/home/aistudio/external-libraries/chinese_calendar')
sys.path.append('/home/aistudio/external-libraries/statsmodels')
sys.path.append('/home/aistudio/external-libraries/plotly')
sys.path.append('/home/aistudio/external-libraries/pmdarima')

In [3]:
import numpy as np
import pandas as pd
import pickle
%matplotlib inline
import matplotlib # 注意这个也要import一次
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import gc

from datetime import date
from datetime import timedelta

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [4]:
# 加载数据
# 仪表能耗
file_name = './work/AI_data/df_meter_daily.pkl'
with open(file_name, 'rb') as f:
    df_meter_daily = pickle.load(f)

# 日期
file_name = './work/AI_data/df_dates.pkl'
with open(file_name, 'rb') as f:
    df_dates = pickle.load(f)

# 每日天气
file_name = './work/AI_data/weather_daily.pkl'
with open(file_name, 'rb') as f:
    weather_daily = pickle.load(f)

# 每小时天气
file_name = './work/AI_data/weather_hour.pkl'
with open(file_name, 'rb') as f:
    weather_hour = pickle.load(f)

# 每日+每小时天气
file_name = './work/AI_data/df_weather_dt.pkl'
with open(file_name, 'rb') as f:
    df_weather_dt = pickle.load(f)

# 每日客流
file_name = './work/AI_data/df_area_custflow.pkl'
with open(file_name, 'rb') as f:
    df_area_custflow = pickle.load(f)

df_meter_daily.shape, df_dates.shape, weather_daily.shape, weather_hour.shape, df_weather_dt.shape,\
df_area_custflow.shape

((625, 243), (90, 273), (301, 243), (7224, 243), (7525, 243), (3416, 243))

In [5]:
# df_meter_daily

In [6]:
meter_index = df_meter_daily.index
df_meter_daily.index = df_meter_daily.index.droplevel(level=[1,2,3,4,5])

In [7]:
no_need_pred_meter = np.all(df_meter_daily.iloc[:, -7::] <= 0.0001, axis=1)

df_noneed = pd.DataFrame(np.zeros([no_need_pred_meter.sum(), 7]),
             index=df_meter_daily[no_need_pred_meter].index)

# df_meter_daily[no_need_pred_meter].to_csv("./test.csv")

df_meter_daily = df_meter_daily[np.logical_not(no_need_pred_meter)]

In [8]:
meters = list(df_meter_daily.index)

In [9]:
idxs = []
for ids in df_area_custflow.index:
    idxs.append(ids[0] + '_' + str(ids[1]))
df_area_custflow.index = idxs

In [10]:
dates_cols = list(df_dates.index)
weather_dt_cols = list(df_weather_dt.index)
custflow_cols = list(df_area_custflow.index)

In [11]:
df_meter_daily = df_meter_daily.T
df_dates = df_dates.T
df_weather_dt = df_weather_dt.T
df_area_custflow = df_area_custflow.T

In [12]:
# df_meter_daily

In [13]:
original_data = df_meter_daily.join(df_dates, how='right')#.join(df_weather_dt, how='left').join(df_area_custflow, how='left')

#### SARIMAX 模型

In [14]:
import pmdarima as pm

In [15]:
# ARIMA模型评价
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
    me = np.mean(forecast - actual)
    mae = np.mean(np.abs(forecast - actual))
    mpe = np.mean((forecast - actual)/actual)
    # rmse = np.mean((forecast - actual)**2)**0.5    # RMSE
    rmse_1 = np.sqrt(sum((forecast - actual) ** 2) / actual.size)
    corr = np.corrcoef(forecast, actual)[0, 1]
    mins = np.amin(np.hstack([forecast[:, None], actual[:, None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:, None], actual[:, None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)
    return ({'mape': mape,
             'me': me,
             'mae': mae,
             'mpe': mpe,
             'rmse': rmse_1,
             'corr': corr,
             'minmax': minmax
             })

In [16]:
def train_test(data, test_size = 0.15, scale = False, cols_to_transform=None, include_test_scale=False):
    """
    
        Perform train-test split with respect to time series structure
        
        - df: dataframe with variables X_n to train on and the dependent output y which is the column 'SDGE' in this notebook
        - test_size: size of test set
        - scale: if True, then the columns in the -'cols_to_transform'- list will be scaled using StandardScaler
        - include_test_scale: If True, the StandardScaler fits the data on the training as well as the test set; if False, then
          the StandardScaler fits only on the training set.
        
    """
    df = data.copy()
    # get the index after which test set starts
    test_index = int(len(df)*(1-test_size))
    
    # StandardScaler fit on the entire dataset
    if scale and include_test_scale:
        scaler = StandardScaler()
        df[cols_to_transform] = scaler.fit_transform(df[cols_to_transform])
        
    X_train = df.drop('y', axis = 1).iloc[:test_index]
    y_train = df['y'].iloc[:test_index]
    X_test = df.drop('y', axis = 1).iloc[test_index:]
    y_test = df['y'].iloc[test_index:]
    
    # StandardScaler fit only on the training set
    if scale and not include_test_scale:
        scaler = StandardScaler()
        X_train[cols_to_transform] = scaler.fit_transform(X_train[cols_to_transform])
        X_test[cols_to_transform] = scaler.transform(X_test[cols_to_transform])
    
    return X_train, X_test, y_train, y_test

In [17]:
# meters

In [ ]:
arimamodels = dict()
df_preds_7days = pd.DataFrame()
start_day = max(df_meter_daily.index) + timedelta(1)
cnt = 0

cols_to_transform = dates_cols# + weather_dt_cols + custflow_cols 
scaler_X = StandardScaler()
original_data[cols_to_transform] = scaler_X.fit_transform(original_data[cols_to_transform])
original_data['constant'] = 1
cols_to_transform = cols_to_transform + ['constant']

for ids in meters:    
    train_data = original_data[[ids] + cols_to_transform].dropna()
    train_data.rename(columns={ids: 'y'}, inplace=True)
    
    scaler_y = StandardScaler()
    train_data['y'] = scaler_y.fit_transform(train_data['y'].values.reshape(-1,1))
    
    # 切分数据
#     X_train_lag, X_test_lag, y_train_lag, y_test_lag = train_test(train_data, \
#                                                                   test_size = 0.15, scale = False,)
    X_train_lag = train_data.drop('y', axis=1)
    y_train_lag = train_data['y']
    
    built_arimamodel = pm.auto_arima(y_train_lag, #data
                                        d=1, # non-seasonal difference order
                                        start_p=0, # initial guess for p
                                        start_q=0, # initial guess for q
                                        test='adf', # ADF检验确认差分阶数d
                                        max_p=1, # max value of p to test
                                        max_q=1,
                                        exogenous= X_train_lag, #including the exogenous variables
                                        seasonal=True, # is the time series seasonal
                                        m=7, # the seasonal period
                                        D=0, # seasonal difference order
                                        start_P=1, # initial guess for P
                                        start_Q=1, # initial guess for Q
                                        max_P=2, # max value of P to test
                                        max_Q=2, # max value of Q to test
                                        information_criterion='aic', # used to select best model
                                        trace=False, # print results whilst training
                                        error_action='ignore', # ignore orders that don't work
                                        stepwise=True, # apply intelligent order search
                                        )
#     built_arimamodel.fit(original_data, X= X_train_lag)
        
#     pred_list = []
#     for index, row in y_test_lag.to_frame().iterrows():
#         # 输出索引，值
# #         print(index, row)
#         pred_list += [built_arimamodel.predict(n_periods=1, X=X_test_lag.loc[[index]])]
#         # 更新模型，model.update()函数，不断用新观测到的 value 更新模型
#         built_arimamodel.update(row, X=X_test_lag.loc[[index]])
#         # 预测时间序列以外未来的一次
#         # predict_f1 = built_arimamodel.predict(n_periods=1)
#         # print('未来一期的预测需求为：', predict_f1[0])

    pred = built_arimamodel.predict_in_sample(start=len(y_train_lag), end=len(y_train_lag) + 6,
                                              X=original_data.iloc[len(train_data):len(train_data)+7][cols_to_transform])
    pred = scaler_y.inverse_transform(pred)
    pred = pd.DataFrame(pred.reshape(1, -1),index=pd.Index([ids]))

#     # 画图观测实际与测试的对比
#     test_predict = y_test_lag.copy()
#     for x in range(len(test_predict)):
#         test_predict.iloc[x] = pred_list[x]
#     # 模型评价
#     eval_result = forecast_accuracy(test_predict.values, y_test_lag.values)
#     print('{}: {} 模型评价结果{}\n'.format(cnt, ids, eval_result))
#     # 画图显示
#     plt.figure(figsize=(10,5))
#     plt.plot(pd.concat((y_train_lag, y_test_lag), axis=0), 'b-', lw=1, label='True Data', marker='*')
#     plt.plot(test_predict, 'r-', lw=2, label='Prediction', marker='o')
#     plt.plot(pred, 'g-', lw=2, label='Prediction_7days', marker='o')
#     # plt.title('RMSE:{}'.format(eval_result['rmse']))
#     plt.legend(loc='best')
#     plt.grid()  # 生成网格
#     plt.show()

    print(cnt, ':', ids)
    cnt += 1

    arimamodels[ids] = built_arimamodel
    df_preds_7days = df_preds_7days.append(pred)

0 : 10835
1 : 10836
2 : 10842
3 : 10843
4 : 10845
5 : 10846
6 : 10847
7 : 10848
8 : 10849
9 : 10850
10 : 10851
11 : 10855
12 : 10856
13 : 10857
14 : 10858
15 : 10859
16 : 10860
17 : 10863
18 : 10864
19 : 10866
20 : 10867
21 : 10868
22 : 10869
23 : 10870
24 : 10871
25 : 10872
26 : 10876
27 : 10877
28 : 10884
29 : 10890
30 : 10891
31 : 10893
32 : 10897
33 : 10900
34 : 10902
35 : 10903
36 : 10904
37 : 10905
38 : 10906
39 : 10907
40 : 10908
41 : 10909
42 : 10910
43 : 10911
44 : 10912
45 : 10914
46 : 10915
47 : 10919
48 : 10920
49 : 10923
50 : 10930
51 : 10931
52 : 10932
53 : 10933
54 : 10935
55 : 10936
56 : 10937
57 : 10940
58 : 10941
59 : 10942
60 : 10943
61 : 10947
62 : 10949
63 : 10950
64 : 10951
65 : 10952
66 : 10953
67 : 10954
68 : 10955
69 : 10956
70 : 10957
71 : 10958
72 : 10959
73 : 10962
74 : 10964
75 : 10965
76 : 10966
77 : 10967
78 : 10969
79 : 10971
80 : 10972
81 : 10974
82 : 10975
83 : 10976
84 : 10977
85 : 10978
86 : 10979
87 : 10981
88 : 10982
89 : 10984
90 : 10988
91 : 1099

In [20]:
pred

,0,1,2,3,4,5,6
10835,1878.536331,1917.339722,2176.578505,1889.464896,1859.914327,1524.266292,1509.641551


In [21]:
preds_7days

,0,1,2,3,4,5,6
10835,1878.536331,1917.339722,2176.578505,1889.464896,1859.914327,1524.266292,1509.641551


In [19]:
####################################

### 组装输出数据

In [20]:
df_items = pd.DataFrame(zip(*meter_index), index=meter_index.names)
df_items = df_items.T.set_index('meter')

In [21]:
df_preds = df_preds_7days.copy()
# df_preds = (df_preds > 0.001) * df_preds

In [22]:
df_preds.columns = range(7)
df_preds = df_preds.join(df_items, how='left')
df_preds

,0,1,2,3,4,5,6,meter_name,meter_function_explain,energy_item_id,c_name,c_order
10835,1.821902e+03,1.971438e+03,2.320322e+03,2.207650e+03,2.112115e+03,1.891910e+03,1.917834e+03,3D1/4 租户负荷1（L1-L2）,正向有功电能二次侧,EI5201314064,租户负荷,68
10836,1.719102e+03,1.745279e+03,1.884288e+03,1.794486e+03,1.826515e+03,1.714326e+03,1.742440e+03,3D1/5 租户负荷2（B1）,正向有功电能二次侧,EI5201314064,租户负荷,68
10837,-4.171221e-07,-7.569883e-07,-1.104080e-06,-7.472821e-07,-3.905710e-08,8.835784e-09,-1.395906e-07,3D1/6-1 备用 无馈线,正向有功电能二次侧,EI1010207001,备用回路,119
10838,-4.171221e-07,-7.569883e-07,-1.104080e-06,-7.472821e-07,-3.905710e-08,8.835784e-09,-1.395906e-07,3D1/6-2 备用 无馈线,正向有功电能二次侧,EI1010207001,备用回路,119
10839,-4.171221e-07,-7.569883e-07,-1.104080e-06,-7.472821e-07,-3.905710e-08,8.835784e-09,-1.395906e-07,3D1/6-3 备用 无馈线,正向有功电能二次侧,EI1010207001,备用回路,119
...,...,...,...,...,...,...,...,...,...,...,...,...
11568,-4.171221e-07,-7.569883e-07,-1.104080e-06,-7.472821e-07,-3.905710e-08,8.835784e-09,-1.395906e-07,10D2/8-5 擦窗机（备用）,正向有功电能二次侧,EI5201314075,擦窗机（办公）,45
11575,3.571749e+01,3.552222e+01,3.574481e+01,3.559140e+01,3.539293e+01,3.546068e+01,3.541916e+01,10D2/8-4 弱电间,正向有功电能二次侧,EI5201314023,信息机房（办公）,34
11576,1.734541e-02,1.392387e-02,1.954648e-02,1.757748e-02,1.254051e-02,1.230856e-02,8.278760e-03,10D1/10-1 消防水泵（备）,正向有功电能二次侧,EI5201314027,消防水泵（办公）,37
11581,1.074119e-06,9.882972e-07,7.413775e-07,-1.572266e-06,-1.983853e-06,-2.544068e-06,-2.603129e-06,6D2/6-6 ZBAL,正向有功电能二次侧,EI5201314085,超市,124


In [23]:
df_preds = df_preds.groupby(by=['energy_item_id'])[0, 1, 2, 3, 4, 5, 6].agg('sum').reset_index()
df_preds

,energy_item_id,0,1,2,3,4,5,6
0,EI10101010104001,1004.677070,1023.087347,946.423881,937.683869,9.595414e+02,9.807442e+02,9.836947e+02
1,EI10101010105001,46.400947,49.417553,-10.768755,-39.722811,-4.526330e+01,1.636989e+00,4.521678e-01
2,EI10101010106001,61.297170,62.279733,62.369454,61.536440,5.974683e+01,5.852825e+01,6.154330e+01
3,EI10101010107001,51.880393,43.254493,37.501192,39.449042,4.875768e+01,4.931875e+01,4.719817e+01
4,EI10101010109001,-0.000002,-0.000003,-0.000004,-0.000003,-1.562284e-07,3.534314e-08,-5.583624e-07
...,...,...,...,...,...,...,...,...
73,EI5201314079,174.514134,172.882088,166.742204,137.147105,1.600165e+02,1.644928e+02,1.718392e+02
74,EI5201314080,59.425497,59.961205,62.913723,65.830648,5.622558e+01,6.354619e+01,6.367005e+01
75,EI5201314081,3521.355706,3551.213089,3665.382077,3518.732679,3.508784e+03,3.512614e+03,3.528948e+03
76,EI5201314082,272.426629,274.433101,275.220347,279.625428,2.788493e+02,2.857603e+02,2.862321e+02


In [24]:
df_submission = pd.read_excel('../data/1.3 t_research_energyitem.xlsx')
df_submission = df_submission[['c_order', 'c_logic_id', 'c_name', 'c_parent']]

In [25]:
def get_energyitem_level(item):
    if len(df_submission[df_submission['c_logic_id'] == item]) == 0:
        return 0
    else:
        p_item = df_submission[df_submission['c_logic_id'] == item]['c_parent'].values[0]
#         print(item, p_item)
        return 1 + get_energyitem_level(p_item)

In [26]:
def is_leaf(item):
    if len(df_submission[df_submission['c_parent'] == item]) == 0:
        return True
    else:
        return False

In [27]:
df_submission['level'] = df_submission['c_logic_id'].apply(lambda x: get_energyitem_level(x))
df_submission['is_leaf'] = df_submission['c_logic_id'].apply(lambda x: is_leaf(x))

In [28]:
df_submission.sort_values(by='c_order', inplace=True)
df_submission

,c_order,c_logic_id,c_name,c_parent,level,is_leaf
61,0,EI1001,总能耗,-1,1,False
70,1,EI101001,光环中心A座,EI1001,2,False
77,2,EI101030102001,办公公区（办公）,EI101001,3,False
103,3,EI101030409001,强弱电机房（办公）,EI101030102001,4,False
84,4,EI101030410001,信息机房（办公）,EI101030409001,5,True
...,...,...,...,...,...,...
52,120,EI101020601001,物业用电,EI10102001,4,True
100,121,EI101020602001,活动用电,EI10102001,4,True
46,122,EI5201314007,运动秀场,EI10102001,4,True
51,123,EI10105001,其他用电,EI10102001,4,True


In [29]:
df_submission = df_submission.merge(df_preds, how='left', left_on=['c_logic_id'], right_on=['energy_item_id'])
df_submission

,c_order,c_logic_id,c_name,c_parent,level,is_leaf,energy_item_id,0,1,2,3,4,5,6
0,0,EI1001,总能耗,-1,1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,EI101001,光环中心A座,EI1001,2,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,EI101030102001,办公公区（办公）,EI101001,3,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,EI101030409001,强弱电机房（办公）,EI101030102001,4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,EI101030410001,信息机房（办公）,EI101030409001,5,True,EI101030410001,2.488335e+02,244.660280,242.356641,243.771295,2.448573e+02,2.429325e+02,2.460875e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,120,EI101020601001,物业用电,EI10102001,4,True,EI101020601001,1.753015e+02,150.911345,132.056730,126.310028,1.526006e+02,1.831760e+02,1.829940e+02
121,121,EI101020602001,活动用电,EI10102001,4,True,EI101020602001,1.785630e+01,19.614126,16.216379,27.346454,2.105104e+01,2.300295e+01,1.607487e+01
122,122,EI5201314007,运动秀场,EI10102001,4,True,EI5201314007,-8.342441e-07,-0.000002,-0.000002,-0.000001,-7.811419e-08,1.767157e-08,-2.791812e-07
123,123,EI10105001,其他用电,EI10102001,4,True,EI10105001,1.102909e+00,-4.501788,1.390143,-5.543923,-6.114172e+00,-5.593140e+00,-5.321188e+00


In [30]:
# 遍历个非叶子科目，统计能耗
for l in range(df_submission['level'].max() - 1, -1, -1):
    for ids in df_submission[(df_submission['level'] == l) & (df_submission['is_leaf'] == False)].index:
        item = df_submission.iloc[ids]['c_logic_id']
        df_submission.loc[ids, [0,1,2,3,4,5,6]] = df_submission[df_submission['c_parent'] == item][[0,1,2,3,4,5,6]].sum()

In [31]:
df_submission_head = \
pd.concat(\
(pd.DataFrame([['998','998','公共用电','998'],
              ['999','999','商户用电','999']], columns=['c_order', 'c_logic_id', 'c_name', 'c_parent'])
,
(pd.DataFrame(np.concatenate((\
#公共用电=（c_order=2+c_order=31 +c_order=72）
df_submission[df_submission['c_order'] == 2][[0, 1, 2, 3, 4, 5, 6]].values\
+ df_submission[df_submission['c_order'] == 31][[0, 1, 2, 3, 4, 5, 6]].values\
+ df_submission[df_submission['c_order'] == 72][[0, 1, 2, 3, 4, 5, 6]].values\
,
# 商户用电=（c_order=29+c_order=58 +c_order=60 +c_order=124）
df_submission[df_submission['c_order'] == 29][[0, 1, 2, 3, 4, 5, 6]].values\
+ df_submission[df_submission['c_order'] == 58][[0, 1, 2, 3, 4, 5, 6]].values\
+ df_submission[df_submission['c_order'] == 60][[0, 1, 2, 3, 4, 5, 6]].values\
+ df_submission[df_submission['c_order'] == 124][[0, 1, 2, 3, 4, 5, 6]].values\
)))))
, axis=1)

In [32]:
df_submission = pd.concat((df_submission_head, df_submission), axis=0)

In [33]:
df_submission[['c_order', 'c_logic_id', 'c_name', 'c_parent', 0, 1, 2, 3, 4, 5, 6]].to_csv("./submission/submission_SARIMAX_导入之前模型_融入时间.csv", index=None)